<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/detect_chord_coords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Draw Coordenates
import cv2
import numpy as np
from PIL import Image

def process_image(image_path, threshold, pixel_threshold):
    def convert_to_black_and_white(image, threshold):
        grey_image = image.convert("L")
        image_array = np.array(grey_image)
        normalized_image_array = image_array / 255.0
        image_array[normalized_image_array > threshold] = 255
        image_array[normalized_image_array <= threshold] = 0
        return Image.fromarray(image_array)

    def detect_rectangle_edges_full_axis(image, pixel_threshold):
        img_gray = np.array(image.convert('L'))
        img = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
        _, binary = cv2.threshold(img_gray, 128, 255, cv2.THRESH_BINARY_INV)

        top_edge, bottom_edge = 0, binary.shape[0] - 1
        left_edge, right_edge = 0, binary.shape[1] - 1

        for row in range(binary.shape[0]):
            if np.sum(binary[row, :] == 255) >= pixel_threshold:
                top_edge = row
                break

        for row in range(binary.shape[0] - 1, -1, -1):
            if np.sum(binary[row, :] == 255) >= pixel_threshold:
                bottom_edge = row
                break

        for col in range(binary.shape[1]):
            if np.sum(binary[:, col] == 255) >= pixel_threshold:
                left_edge = col
                break

        for col in range(binary.shape[1] - 1, -1, -1):
            if np.sum(binary[:, col] == 255) >= pixel_threshold:
                right_edge = col
                break

        cv2.line(img, (0, top_edge), (img.shape[1], top_edge), (0, 0, 255), 1)
        cv2.line(img, (0, bottom_edge), (img.shape[1], bottom_edge), (0, 0, 255), 1)
        cv2.line(img, (left_edge, 0), (left_edge, img.shape[0]), (255, 0, 0), 1)
        cv2.line(img, (right_edge, 0), (right_edge, img.shape[0]), (255, 0, 0), 1)

        return img

    def process_final_image(image):
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        blue_lower, blue_upper = np.array([0, 0, 100], dtype="uint8"), np.array([50, 50, 255], dtype="uint8")
        red_lower, red_upper = np.array([100, 0, 0], dtype="uint8"), np.array([255, 50, 50], dtype="uint8")

        blue_mask = cv2.inRange(image_rgb, blue_lower, blue_upper)
        red_mask = cv2.inRange(image_rgb, red_lower, red_upper)

        blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        blue_lines_x = sorted([cv2.boundingRect(contour)[0] for contour in blue_contours])
        red_lines_y = sorted(set([cv2.boundingRect(contour)[1] for contour in red_contours]))

        vertical_distance = np.abs(blue_lines_x[0] - blue_lines_x[1])
        horizontal_distance = np.abs(red_lines_y[0] - red_lines_y[1])
        N = horizontal_distance // 4

        # Draw intermediate blue lines
        blue_intermediate_positions = np.linspace(blue_lines_x[0], blue_lines_x[1], 6)
        for x in blue_intermediate_positions:
            cv2.line(image, (int(x), 0), (int(x), image.shape[0]), (255, 0, 0), 1)

        # Draw green horizontal lines
        first_green_line_position = red_lines_y[0] - (N // 2)
        green_line_positions = [first_green_line_position + 10]  # Move first green line 100 pixels lower
        green_line_positions += [first_green_line_position + (i + 1) * N for i in range(4)]  # Other green lines remain the same

        for y in green_line_positions:
            cv2.line(image, (0, int(y)), (image.shape[1], int(y)), (0, 255, 0), 1)

        return image

    # Start of the main function process
    original_image = Image.open(image_path)
    bw_image = convert_to_black_and_white(original_image, threshold)
    edge_detected_image = detect_rectangle_edges_full_axis(bw_image, pixel_threshold)
    final_image = process_final_image(edge_detected_image)

    final_output_path = 'final_processed_image.png'
    cv2.imwrite(final_output_path, final_image)
    return final_output_path

# Example usage
image_path = '/content/A#dim.png'
threshold = 0.5
pixel_threshold = 200
output_path = process_image(image_path, threshold, pixel_threshold)
print(output_path)
